In [2]:
import os
import random
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [3]:
def load_data(img_path, label_path):
    train_image = sorted(glob.glob(os.path.join(img_path, '*.jpg')))
    train_label = pd.read_csv(label_path)
    train_label = train_label.iloc[:,1].values.tolist()
    
    train_data = list(zip(train_image, train_label))
    random.shuffle(train_data)
    
    train_set = train_data[:20000]
    valid_set = train_data[20000:]
    
    return train_set, valid_set
    

In [4]:
class hw3_dataset(Dataset):
    
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx][0])
        img = self.transform(img)
        label = self.data[idx][1]
        return img, label

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.LeakyReLU(negative_slope=0.05),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),     
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.05),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),            
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3,padding=1),
            nn.LeakyReLU(negative_slope=0.05),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3,padding=1),
            nn.LeakyReLU(negative_slope=0.05),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            
        )
        self.fc = nn.Sequential(
            nn.Linear(3*3*128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 7)
        )

    def forward(self, x):
        #image size (48,48)
        x = self.conv1(x) #(24,24)
        x = self.conv2(x) #(12,12)
        x = self.conv3(x) #(6,6)
        x = self.conv4(x) #(3,3)
        x = x.view(-1, 3*3*128)
        x = self.fc(x)
        return x

In [9]:
if __name__ == '__main__':
    use_gpu = torch.cuda.is_available()

    train_set, valid_set = load_data('./train_img', 'train.csv')

    #transform to tensor, data augmentation
    
    transform = transforms.Compose([
    #transforms.RandomAffine(15, translate=(0.1,0.1), scale=(0.9,1.1), shear=10, fillcolor=0),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize([mean], [std], inplace=False)
    ])
    
    train_dataset = hw3_dataset(train_set,transform)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    valid_dataset = hw3_dataset(valid_set,transform)
    valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
    """
    model = Net()
    if use_gpu:
        model.cuda()
    """
    model =Net()
    model.load_state_dict(torch.load('model_10.pth'))
    print("load model")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()

    num_epoch = 50
    for epoch in range(num_epoch):
        model.train()
        train_loss = []
        train_acc = []
        for idx, (img, label) in enumerate(train_loader):
            if use_gpu:
                img = img.cuda()
                label = label.cuda()
            optimizer.zero_grad()
            output = model(img)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()

            predict = torch.max(output, 1)[1]
            acc = np.mean((label == predict).cpu().numpy())
            train_acc.append(acc)
            train_loss.append(loss.item())
        print("Epoch: {}, train Loss: {:.4f}, train Acc: {:.4f}".format(epoch + 1, np.mean(train_loss), np.mean(train_acc)))


        model.eval()
        with torch.no_grad():
            valid_loss = []
            valid_acc = []
            for idx, (img, label) in enumerate(valid_loader):
                if use_gpu:
                    img = img.cuda()
                    label = label.cuda()
                output = model(img)
                loss = loss_fn(output, label)
                predict = torch.max(output, 1)[1]
                acc = np.mean((label == predict).cpu().numpy())
                valid_loss.append(loss.item())
                valid_acc.append(acc)
            print("Epoch: {}, valid Loss: {:.4f}, valid Acc: {:.4f}".format(epoch + 1, np.mean(valid_loss), np.mean(valid_acc)))
        
        if np.mean(train_acc) > 0.9:
            checkpoint_path = './ta_model/model_{}.pth'.format(epoch+1) 
            torch.save(model.state_dict(), checkpoint_path)
            print('model saved to %s' % checkpoint_path)
    

    #finish test code

load model
Epoch: 1, train Loss: 0.6642, train Acc: 0.8063
Epoch: 1, valid Loss: 0.6971, valid Acc: 0.7879
Epoch: 2, train Loss: 0.3471, train Acc: 0.8912
Epoch: 2, valid Loss: 0.6882, valid Acc: 0.7965
Epoch: 3, train Loss: 0.1779, train Acc: 0.9477
Epoch: 3, valid Loss: 0.7841, valid Acc: 0.7872
model saved to model_3.pth
Epoch: 4, train Loss: 0.1011, train Acc: 0.9730
Epoch: 4, valid Loss: 0.9268, valid Acc: 0.7586
model saved to model_4.pth
Epoch: 5, train Loss: 0.1029, train Acc: 0.9695
Epoch: 5, valid Loss: 1.1556, valid Acc: 0.7230
model saved to model_5.pth
Epoch: 6, train Loss: 0.1406, train Acc: 0.9533
Epoch: 6, valid Loss: 1.0881, valid Acc: 0.7305
model saved to model_6.pth
Epoch: 7, train Loss: 0.1417, train Acc: 0.9523
Epoch: 7, valid Loss: 1.1121, valid Acc: 0.7305
model saved to model_7.pth
Epoch: 8, train Loss: 0.1052, train Acc: 0.9662
Epoch: 8, valid Loss: 1.4067, valid Acc: 0.6786
model saved to model_8.pth
Epoch: 9, train Loss: 0.0653, train Acc: 0.9811
Epoch: 9, v

In [ ]:
if __name__ == '__main__':
    #......